<a href="https://colab.research.google.com/github/DEP04929/ESICMDatathon2026/blob/main/ExtractingData_20260114.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pre-requisites for Amsterdam UMC DB


In [2]:
# sets *your* project id
PROJECT_ID = "esicmdatathon2026" #@param {type:"string"}


In [3]:
# sets default dataset for AmsterdamUMCdb
DATASET_PROJECT_ID = 'amsterdamumcdb' #@param {type:"string"}
DATASET_ID = 'van_gogh_2026_datathon' #@param {type:"string"}
LOCATION = 'eu' #@param {type:"string"}

In [4]:
import os
from google.colab import auth

# all libraries check this environment variable, so set it:
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID

auth.authenticate_user()
print('Authenticated')


Authenticated


In [5]:
%load_ext google.colab.data_table
from google.colab.data_table import DataTable

# change default limits:
DataTable.max_columns = 50
DataTable.max_rows = 50000


In [6]:
from google.cloud.bigquery import magics
from google.cloud import bigquery

# sets the default query job configuration
def_config = bigquery.job.QueryJobConfig(default_dataset=DATASET_PROJECT_ID + "." + DATASET_ID)
magics.context.default_query_job_config = def_config


In [7]:
import pandas as pd
import numpy as np

import matplotlib as plt
import seaborn as sns
sns.set_style('darkgrid')

# all possible ventilator patients
this will be used as criteria of person ids for all subsequent extractions

In [81]:
%%bigquery ventpatlist --project $PROJECT_ID
select distinct person_id
from measurement m inner join concept c on m.measurement_concept_id = c.concept_id
where concept_name like '%ventila%'

Query is running:   0%|          |

Downloading:   0%|          |

# ? Diagnosis: surgical or non surgical
All entries in condition occurrence are admission diagnosis. They are uncoded, so free text search is necessary.


In [104]:
%%bigquery surg --project esicmdatathon2026
with vlist as (
  select distinct person_id
from measurement m inner join concept c on m.measurement_concept_id = c.concept_id
where concept_name like '%ventila%'
), surg as (
select person_id, condition_start_datetime, condition_source_value
, case when lower(condition_source_value) like '%non%surg%' then 'non-surgical' else 'surgical' end as condition_type
from condition_occurrence
where condition_source_value like '%surg%' --matches surgical and non surgical
  and person_id in (select person_id from vlist)
)
select * from surg
PIVOT ( count(*) for condition_type in ('surgical', 'non-surgical'))

Query is running:   0%|          |

Downloading:   0%|          |

In [105]:
surg.head(20)

,person_id,condition_start_datetime,condition_source_value,surgical,non-surgical
0,47916,2001-12-31 17:35:00+00:00,"Esophageal surgery, other",1,0
1,46368,2001-12-31 18:04:00+00:00,Pulmonary valve surgery,1,0
2,50290,2001-12-31 18:20:00+00:00,"Esophageal surgery, other",1,0
3,51278,2001-12-31 18:38:00+00:00,"Esophageal surgery, other",1,0
4,27093,2001-12-31 19:17:00+00:00,"Esophageal surgery, other",1,0
5,32519,2001-12-31 19:28:00+00:00,"Esophageal surgery, other",1,0
6,7623,2001-12-31 20:10:00+00:00,"Abdomen only trauma, surgery for",1,0
7,50776,2001-12-31 21:31:00+00:00,"GI Obstruction, surgery for (including lysis o...",1,0
8,44693,2002-01-01 00:00:00+00:00,Pulmonary embolus - Non surgical,0,1
9,16284,2002-01-01 00:01:00+00:00,Asthma or allergy - Non surgical,0,1


In [106]:
top_10_conditions = surg.groupby('condition_source_value')['person_id'].nunique().nlargest(10)
print(top_10_conditions)

condition_source_value
Heart valve surgery - Surgical                                       2397
Infection - Non surgical                                              922
Respiratory - Non surgical                                            862
After cardiac arrest - surgical Ã« Non surgical                       842
Peripheral vascular surgery - Surgical                                769
Neurological - Non surgical                                           672
Intracerebral subdural or subarachnoid haemorrhage - Non surgical     651
Sepsis - Surgical Ã« non surgical                                     613
Cardiovascular surgery, other                                         466
Gastrointestinal - Non surgical                                       455
Name: person_id, dtype: int64


# Heart rate

In [109]:
%%bigquery hf_ekg --project esicmdatathon2026
with vlist as (
  select distinct person_id
from measurement m inner join concept c on m.measurement_concept_id = c.concept_id
where concept_name like '%ventila%'
)
select person_id, measurement_datetime, value_as_number as hr_ekg
from measurement m
where person_id in (select person_id from vlist)
  and measurement_concept_id in (
	21490872 --Heart rate.beat-to-beat by EKG
)

Query is running:   0%|          |

Downloading:   0%|          |

This shows the full range of all heart rate possibilities

In [110]:
hf_ekg.head(50)


,person_id,measurement_datetime,hr_ekg
0,65352,2009-12-31 21:00:00+00:00,120.0
1,65354,2019-09-09 23:38:00+00:00,19.0
2,65354,2019-09-09 23:42:00+00:00,22.0
3,65354,2019-09-09 23:37:00+00:00,22.0
4,65354,2019-09-09 23:39:00+00:00,22.0
5,65354,2019-09-10 01:26:00+00:00,29.0
6,65354,2019-09-09 23:36:00+00:00,30.0
7,65354,2019-09-09 23:40:00+00:00,30.0
8,65354,2019-09-08 19:03:00+00:00,111.0
9,65354,2019-09-04 23:33:00+00:00,111.0


# Arterial blood pressure

In [125]:
%%bigquery bp --project $PROJECT_ID
with vlist as (
  select distinct person_id
from measurement m inner join concept c on m.measurement_concept_id = c.concept_id
where concept_name like '%ventila%'
)
, bp as (
select person_id, measurement_datetime, value_as_number
, case when measurement_concept_id = 21490853 then 'ABPs'
     when measurement_concept_id = 21490851 then 'ABPd'
     when measurement_concept_id = 21490852 then 'ABPm'
--     when measurement_concept_id = 21492241 then 'NiBPm'
     end as measure_type
from measurement m
where person_id in (select person_id from vlist)
  and measurement_concept_id in ( 21490851 -- ABP d
  , 21490852 -- ABP m
  , 21490853 -- ABPs
--  , 21492241 -- NiBPm
)
)
select * from bp
PIVOT ( max(value_as_number) for measure_type in ('ABPs', 'ABPd', 'ABPm')) --, 'NiBPm'


Query is running:   0%|          |

Downloading:   0%|          |

In [126]:
bp

,person_id,measurement_datetime,ABPs,ABPd,ABPm
0,30,2002-01-02 13:29:00+00:00,96.000018,40.999996,62.000002
1,332,2002-01-06 20:31:00+00:00,122.000005,56.000003,80.000002
2,30,2002-01-14 22:29:00+00:00,143.999987,62.000002,84.999991
3,174,2010-01-15 00:20:00+00:00,147.999999,53.000001,80.000002
4,663,2010-08-23 13:55:00+00:00,87.000026,53.000001,69.000003
...,...,...,...,...,...
8166435,23635,2017-01-05 08:35:00+00:00,165.000000,65.000000,101.000000
8166436,39725,2016-12-31 20:36:00+00:00,161.000000,NaN,101.000000
8166437,38749,2016-12-31 14:43:00+00:00,147.000000,77.000000,101.000000
8166438,39029,2017-01-06 02:11:00+00:00,140.000000,79.000000,101.000000


# Breath rate or respiratory rate

Breath rate from ventilator and respiratory rate from monitor gets mixed up over the years of data recording.

In [10]:
%%bigquery rr --project esicmdatathon2026
with vlist as (
  select distinct person_id
from measurement m inner join concept c on m.measurement_concept_id = c.concept_id
where concept_name like '%ventila%'
)
, rr as (
select person_id, measurement_datetime, value_as_number
, case when measurement_concept_id = 3043148 then 'BRvent'
     when measurement_concept_id = 3026892 then 'BRspontv'
     when measurement_concept_id = 1175625 then 'BRspont'
     when measurement_concept_id = 3024171 then 'RR'
     end as measure_type
from measurement m
where person_id in (select person_id from vlist)
  and measurement_concept_id in ( 3043148 --Breath rate mechanical
                                 , 3026892 -- Breath rate spontaneous --on ventilator
                                 , 1175625 -- Breath rate spontaneous
                                 , 3024171 -- Respiratory rate
                                 )
)
select * from rr
PIVOT ( max(value_as_number) for measure_type in ('BRvent', 'BRspontv','BRspont', 'RR'))


Query is running:   0%|          |

Downloading:   0%|          |

In [14]:
# rr[rr['BRvent']>90] # plausibility check
rr[rr['person_id']==23444] # testing purpose, random person


,person_id,measurement_datetime,BRvent,BRspontv,BRspont,RR
1253,23444,2010-01-15 16:11:00+00:00,NaN,NaN,NaN,30.100000
1421,23444,2010-01-07 12:11:00+00:00,NaN,NaN,NaN,36.200001
3530,23444,2010-01-16 20:11:00+00:00,NaN,NaN,NaN,12.300000
3935,23444,2010-01-06 19:11:00+00:00,NaN,NaN,NaN,28.700001
5859,23444,2010-01-16 18:11:00+00:00,NaN,NaN,NaN,24.600000
...,...,...,...,...,...,...
12051762,23444,2010-01-11 00:19:00+00:00,204.800003,NaN,NaN,NaN
12445208,23444,2010-01-03 12:11:00+00:00,NaN,NaN,NaN,20.000000
12454551,23444,2010-01-03 17:11:00+00:00,NaN,NaN,NaN,20.000000
12468836,23444,2010-01-03 00:11:00+00:00,NaN,NaN,NaN,21.000000


# BGA?

# Lactate

In [26]:
%%bigquery lac --project esicmdatathon2026
--select person_id, measurement_datetime, value_as_number as lactate
select measurement_concept_id, concept_name, measurement_source_value, count(value_as_number)
FROM measurement m inner join concept c on m.measurement_concept_id = c.concept_id
where concept_name like '%Lactate%'
 -- WHERE
 --   measurement_concept_id in (3018405, 3047181, 3028271, 3005949, 3008037) -- searching for lactate
 --  and m.person_id = 56647 -- testing purpose, random visit
 group by measurement_concept_id, concept_name, measurement_source_value
 order by count(value_as_number) desc
limit 100


Query is running:   0%|          |

Downloading:   0%|          |

In [27]:
lac


,measurement_concept_id,concept_name,measurement_source_value,f0_
0,3018405,Lactate [Moles/volume] in Arterial blood,TESTUITSLAG ZONDER AANVRAAG AMC;LACTAAT - BLOE...,224801
1,3047181,Lactate [Moles/volume] in Blood,Lactaat (bloed),180787
2,3018405,Lactate [Moles/volume] in Arterial blood,LACTAAT POCT;LACTAAT ART. VUMC,166011
3,3018405,Lactate [Moles/volume] in Arterial blood,POCT BLOED - BLOEDGAS;LACTAAT ART. VUMC,120627
4,3016436,Lactate dehydrogenase [Enzymatic activity/volu...,LD (bloed),65750
...,...,...,...,...
95,3016436,Lactate dehydrogenase [Enzymatic activity/volu...,VITAMINE B12;LDH,2
96,3016436,Lactate dehydrogenase [Enzymatic activity/volu...,A1-FOETOPROTE�NE (AFP);LDH,2
97,3047181,Lactate [Moles/volume] in Blood,HEMOGLOBINE (BLOEDGAS);LACTAAT - BLOEDGAS,2
98,3016436,Lactate dehydrogenase [Enzymatic activity/volu...,MAGNESIUM;LDH,2


# PEEP

In [33]:
%%bigquery peep --project $PROJECT_ID
with vlist as (
  select distinct person_id
from measurement m inner join concept c on m.measurement_concept_id = c.concept_id
where concept_name like '%ventila%'
),
 vent as
( select person_id, measurement_datetime
  , case when measurement_concept_id = 3012410 then 'TV'
       when measurement_concept_id = 2000000222 then 'MVspont'
       when measurement_concept_id = 3022875 then 'PEEP'
       end as concept_name
  , value_as_number
  from measurement m
  where measurement_concept_id in (
        3012410 -- Tidal volume setting Ventilator
        , 3022875 -- Positive end expiratory pressure setting Ventilator
        , 2000000222 -- Minute volume spontaneous
  )
  and value_as_number is not null
  and person_id in (select person_id from vlist)
)
select * from vent
  PIVOT (max(value_as_number) for concept_name in ('MVspont','TV', 'PEEP'))


Query is running:   0%|          |

Downloading:   0%|          |

In [34]:
peep


,person_id,measurement_datetime,MVspont,TV,PEEP
0,131,2010-01-08 00:15:00+00:00,NaN,666.0,4.0
1,299,2012-01-21 18:56:00+00:00,NaN,455.0,5.0
2,49,2012-03-29 19:26:00+00:00,NaN,397.0,5.0
3,499,2016-11-13 16:42:00+00:00,NaN,472.0,5.0
4,488,2016-11-24 02:33:00+00:00,NaN,645.0,8.0
...,...,...,...,...,...
8121730,32205,2016-12-31 15:55:00+00:00,NaN,600.0,NaN
8121731,6071,2016-12-19 21:28:00+00:00,NaN,600.0,NaN
8121732,9689,2016-12-31 14:43:00+00:00,NaN,600.0,NaN
8121733,9689,2016-12-31 14:38:00+00:00,NaN,600.0,NaN


# FiO2

FiO2 exists for NIV too.

In [43]:
%%bigquery FiO2 --project esicmdatathon2026
with vlist as (
  select distinct person_id
from measurement m inner join concept c on m.measurement_concept_id = c.concept_id
where concept_name like '%ventila%'
),
fio2 as (
select person_id, measurement_datetime
, case when measurement_concept_id in ( 3025408, 2000000204) then 'FiO2'
       when measurement_concept_id = 2000000203 then 'FiO2niv'
       end as concept_name
, value_as_number
from measurement m
where measurement_concept_id in (2000000203 -- FiO2 NIV
  , 3025408 -- Oxygen/Inspired gas Respiratory system by O2 Analyzer --on ventilator
  --, 2000000204 -- fio2 setting
)
  and person_id in (select person_id from vlist)
  and value_as_number > 0
)
select * from fio2
PIVOT (max(value_as_number) for concept_name in ('FiO2', 'FiO2niv'))


Query is running:   0%|          |

Downloading:   0%|          |

In [44]:
FiO2

,person_id,measurement_datetime,FiO2,FiO2niv
0,16014,2017-01-24 19:56:00+00:00,NaN,21.0
1,26770,2017-01-21 06:08:00+00:00,NaN,21.0
2,31775,2017-02-02 18:48:00+00:00,NaN,21.0
3,47214,2011-10-07 16:12:00+00:00,NaN,21.0
4,51243,2017-01-22 15:01:00+00:00,NaN,21.0
...,...,...,...,...
710481,16283,2017-01-01 11:42:00+00:00,100.0,NaN
710482,12940,2017-01-04 18:58:00+00:00,100.0,NaN
710483,12086,2017-01-07 16:22:00+00:00,100.0,NaN
710484,7394,2017-01-10 12:31:00+00:00,100.0,NaN


# CAM score?

In [ ]:
%%bigquery test --project esicmdatathon2026
select *
from observation
where observation_concept_id in (3042082 --Confusion Assessment Method (CAM)
)
limit 100

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
test

,person_id,observation_concept_id,observation_date,observation_datetime,observation_type_concept_id,value_as_number,observation_source_value,unit_source_value,value_source_value
0,13316,3042082,2006-08-07,2006-08-07 04:33:00+00:00,0,0.0,CAM-ICU fluctuerend beloop,,0.0
1,13316,3042082,2006-08-07,2006-08-07 04:33:00+00:00,0,0.0,CAM-ICU fluctuerend beloop,,0.0
2,13316,3042082,2006-08-07,2006-08-07 16:33:00+00:00,0,0.0,CAM-ICU fluctuerend beloop,,0.0
3,13316,3042082,2006-08-07,2006-08-07 16:33:00+00:00,0,0.0,CAM-ICU fluctuerend beloop,,0.0
4,57364,3042082,2006-12-10,2006-12-10 05:51:00+00:00,0,0.0,CAM-ICU fluctuerend beloop,,0.0
...,...,...,...,...,...,...,...,...,...
95,14704,3042082,2007-01-04,2007-01-04 04:27:00+00:00,0,0.0,CAM-ICU fluctuerend beloop,,0.0
96,14704,3042082,2007-01-04,2007-01-04 16:27:00+00:00,0,0.0,CAM-ICU fluctuerend beloop,,0.0
97,14704,3042082,2007-01-04,2007-01-04 16:27:00+00:00,0,0.0,CAM-ICU fluctuerend beloop,,0.0
98,14704,3042082,2007-01-04,2007-01-04 16:27:00+00:00,0,0.0,CAM-ICU fluctuerend beloop,,0.0


In [ ]:
%%bigquery test --project esicmdatathon2026
select person_id, measurement_date, stddev_samp(value_as_number) as sd
from measurement m
inner join concept c
on m.measurement_concept_id = c.concept_id
where measurement_concept_id in ( 21490872 --Heart rate.beat-to-beat by EKG
) and value_as_number > 0 and value_as_number < 300
group by person_id, measurement_date

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
%%bigquery test --project $PROJECT_ID
select c.concept_name, m.measurement_source_value
, max(measurement_date) as last_date_used
, count(distinct person_id) as person_count
, count(*) as number_of_entries
from measurement m
inner join concept c
on m.measurement_concept_id = c.concept_id
where measurement_concept_id = 3024171
group by concept_name, measurement_source_value
order by count(*) desc



Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
test

,concept_name,measurement_source_value,last_date_used,person_count,number_of_entries
0,Respiratory rate,AF totaal;R UMCA AN AF TOTAAL,2023-10-16,17485,6119517
1,Respiratory rate,AF (monitor);R UMCA ICU ADEMFREQUENTIE MONITOR,2022-06-16,21543,2377658
2,Respiratory rate,Respiratory Rate;R RESP RATE,2023-02-05,6349,2042237
3,Respiratory rate,Ademfreq.;ADEMHALING,2023-11-29,22390,1348806
4,Respiratory rate,Ademfreq.,2015-06-12,14263,1271302
5,Respiratory rate,AF (meet);R UMCA AN AF GEMETEN,2023-09-03,3286,1035294
6,Respiratory rate,AF prehosp.;R ED PRE-ARRIVAL RESP RATE,2023-07-01,1977,2296
7,Respiratory rate,Frequentie ingesteld;R UMCA NIV FREQUENTIE S,2021-03-01,83,2045
8,Respiratory rate,Frequentie gemeten;R UMCA NIV FREQUENTIE GEMETEN,2019-11-08,67,1688
9,Respiratory rate,Rate (Bipap Vision),2002-02-01,17,210


# Drugs?

Quantity is there, but dose is not there, for drug exposure. I am unsure mg/ 24 hours unit meaning?  

In [ ]:
%%bigquery test --project esicmdatathon2026
select * --person_id, drug_exposure_start_datetime, drug_exposure_end_datetime, drug_concept_id, quantity
from drug_exposure
where drug_concept_id in (1736887 -- linezolid
                      ,1709170    -- Meropenem
                      ,1707687    -- vancomycin
                      ,1742253    -- Levofloxacin
                      ,1778162    -- Cefuroxim
                      ,1707164    -- Metronidazol
                      ,1777806    -- Ceftriaxone
                      ,1718054    -- Caspofungin
                      ,1754994    -- Fluconazole
                      ,997881     -- Clindamycin
                      )
      and person_id = 56647 -- testing only
  limit 100

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
test

,person_id,drug_concept_id,drug_exposure_start_date,drug_exposure_start_datetime,drug_exposure_end_date,drug_exposure_end_datetime,drug_type_concept_id,quantity,drug_source_value,route_source_value,dose_unit_source_value
0,56647,1777806,2016-12-30,2016-12-30 20:25:00+00:00,2016-12-30,2016-12-30 20:25:00+00:00,38000180,2000.0,CEFTRIAXON 100 MG/ML INJECTIE,intraveneus,mg
1,56647,1742253,2017-01-08,2017-01-08 14:34:00+00:00,2017-01-08,2017-01-08 14:34:00+00:00,38000180,500.0,LEVOFLOXACINE KORTLOPEND INFUUS,intraveneus,mg
2,56647,1742253,2017-01-09,2017-01-09 00:07:00+00:00,2017-01-09,2017-01-09 00:07:00+00:00,38000180,125.0,LEVOFLOXACINE KORTLOPEND INFUUS,intraveneus,mg
3,56647,1742253,2017-01-09,2017-01-09 11:55:00+00:00,2017-01-09,2017-01-09 11:55:00+00:00,38000180,125.0,LEVOFLOXACINE KORTLOPEND INFUUS,intraveneus,mg
4,56647,1742253,2017-01-10,2017-01-10 00:54:00+00:00,2017-01-10,2017-01-10 00:54:00+00:00,38000180,500.0,LEVOFLOXACINE KORTLOPEND INFUUS,intraveneus,mg
...,...,...,...,...,...,...,...,...,...,...,...
72,56647,1707687,2017-01-10,2017-01-10 22:02:00+00:00,2017-01-10,2017-01-10 22:02:00+00:00,38000180,300.0,VANCOMYCINE INFUUS >250 MG EN <=500 MG IN NACL...,intraveneus,mg
73,56647,1709170,2017-01-11,2017-01-11 10:07:00+00:00,2017-01-11,2017-01-11 10:07:00+00:00,38000180,1000.0,MEROPENEM KORTLOPEND INFUUS IN NACL (VUMC),intraveneus,mg
74,56647,1707687,2017-01-11,2017-01-11 04:33:00+00:00,2017-01-11,2017-01-11 04:33:00+00:00,38000180,1000.0,VANCOMYCINE INFUUS >500 MG EN <=1250 MG IN NAC...,intraveneus,mg
75,56647,1707687,2017-01-28,2017-01-28 05:21:00+00:00,2017-01-28,2017-01-28 05:21:00+00:00,38000180,1000.0,VANCOMYCINE INFUUS >500 MG EN <=1250 MG IN NAC...,intraveneus,mg
